In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [13]:
from bs4 import BeautifulSoup

In [14]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

In [15]:
# Create a data frame from BeautifulSoup object soup
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
Toronto_df=df
Toronto_df.describe()

,Postcode,Borough,Neighbourhood
count,288,288,288
unique,180,12,209
top,M9V,Not assigned,Not assigned
freq,8,77,78


In [16]:
#To see which values are present in a Borough column 
Toronto_df['Borough'].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [17]:

# Drop whole row with 'Not assigned' in "Borough" column
Toronto_df1=Toronto_df[Toronto_df.Borough != 'Not assigned']

# reset index, because we droped two rows
Toronto_df1.reset_index(drop=True, inplace=True)

#Replace Not assigned value with Borough for Neighbourhood
Toronto_df1.Neighbourhood.replace("Not assigned",Toronto_df1.Borough,inplace=True)

Toronto_df1['Borough'].value_counts()

Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [18]:
#More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, 
#you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma

Toronto_df2 = Toronto_df1.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
#Reset index to Postcode and Borough
Toronto_df2.reset_index(level=['Postcode','Borough'], inplace=True)
Toronto_df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
#rename columns 
Toronto_df2.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)

In [21]:
Toronto_df2.shape

(103, 3)